In [2]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from tensorflow.keras.utils import to_categorical # one-hot keras encording
from keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import seaborn as sns

# seed 값 설정
seed = 0
np.random.seed(3)
tf.random.set_seed(3)

In [8]:
df = pd.read_csv('C:/Users/user/ILIFO/DeepLearning/deeplearning_class-master/dataset/bike_sharing_demand_train.csv')
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [11]:
df['datetime'] = df['datetime'].astype('datetime64')

In [12]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

In [16]:
df = df.drop(['datetime'],axis=1)

KeyError: "['datetime'] not found in axis"

In [17]:
df

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012,12,19,19
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012,12,19,20
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012,12,19,21
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012,12,19,22


In [18]:
dataset = df.values

In [19]:
dataset

array([[ 1.,  0.,  0., ...,  1.,  1.,  0.],
       [ 1.,  0.,  0., ...,  1.,  1.,  1.],
       [ 1.,  0.,  0., ...,  1.,  1.,  2.],
       ...,
       [ 4.,  0.,  1., ..., 12., 19., 21.],
       [ 4.,  0.,  1., ..., 12., 19., 22.],
       [ 4.,  0.,  1., ..., 12., 19., 23.]])

In [20]:
X = dataset[:,[0,1,2,3,4,5,6,7,14]]
Y = dataset[:,10]

In [22]:
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [23]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size = 0.2)

In [24]:
model = Sequential()

# 입력데이터 17개의 값을 받아 은닉층 30개 노드로 보낸다
model.add(Dense(30, input_dim=9, activation='relu'))
model.add(Dense(25, input_dim=30, activation='relu'))
model.add(Dense(20, input_dim=25, activation='relu'))
model.add(Dense(15, input_dim=20, activation='relu'))
model.add(Dense(10, input_dim=15, activation='relu'))
model.add(Dense(5, input_dim=10, activation='relu'))
model.add(Dense(1))

In [25]:
model.compile(loss='mean_squared_error',
          optimizer='adam',
          metrics=['accuracy'])

In [ ]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=1000)

h = model.fit(x_train, y_train, validation_split= 0.2, epochs=1000000, batch_size=500, verbose=0, callbacks=[early_stopping_callback,checkpointer])


Epoch 00001: val_loss improved from inf to 71079.64062, saving model to ./model/01-71079.6406.hdf5

Epoch 00002: val_loss improved from 71079.64062 to 71020.92969, saving model to ./model/02-71020.9297.hdf5

Epoch 00003: val_loss improved from 71020.92969 to 70992.09375, saving model to ./model/03-70992.0938.hdf5

Epoch 00004: val_loss improved from 70992.09375 to 70974.12500, saving model to ./model/04-70974.1250.hdf5

Epoch 00005: val_loss improved from 70974.12500 to 70964.79688, saving model to ./model/05-70964.7969.hdf5

Epoch 00006: val_loss improved from 70964.79688 to 70958.57812, saving model to ./model/06-70958.5781.hdf5

Epoch 00007: val_loss improved from 70958.57812 to 70952.88281, saving model to ./model/07-70952.8828.hdf5

Epoch 00008: val_loss improved from 70952.88281 to 70947.35156, saving model to ./model/08-70947.3516.hdf5

Epoch 00009: val_loss improved from 70947.35156 to 70941.92969, saving model to ./model/09-70941.9297.hdf5

Epoch 00010: val_loss improved from

In [ ]:
plt.plot(h.history['loss'],'b')
plt.plot(h.history['val_loss'],'r')
plt.show()

In [ ]:
h.history['loss'][-1]